In [ ]:
# import torch
# from transformers import AutoModelForSequenceClassification, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
# model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
# model.eval()

# pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
# with torch.no_grad():
#     inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=1024)
#     scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    

In [1]:
import requests
from openai import OpenAI
import pandas as pd
import re

In [2]:
import pickle

with open('voyage.pkl', 'rb') as f:
    data = pickle.load(f)

In [6]:
Errors = 0
page_check = 0
tokens = 0
top_k = 10
correct = 0
failure = 0
counter = 0
results = []

for d in data:
    query = d['Q']
    answer = d['A']
    retrived_pages = []
    retrived_indexes = d['question_retrived_page_rank'][:top_k]
    retrived_indexes = sorted([list(retrived_indexes[u].keys())[0] for u in range(len(retrived_indexes))])
    
    for x in data:
        if(int(x['page']) in retrived_indexes):
            # retrived_pages += f"This is page {int(x['page'])}: " + x['content'] + "\n"
            retrived_pages.append({"page": x['page'], "content": x['content']})


    # rerank
    try:
        client = OpenAI(api_key='OPENAI_API_KEY_PLACEHOLDER')
        r = client.chat.completions.create(
            messages=[
                {
                "role": "system", "content": "You are a math expert",
                "content": "user", "content": f"I am giving you a question and a some pages in the textbook. The question may or may not be based on one of the pages. Return ONLY a JSON list of page number such that the one coming first is most relevant to the question, and the last is least relevant. You MUST RETURN ONLY JSON LIST. I am giving you {top_k} pages, and therefore expect a list of the same length. Pages:" + str(retrived_pages) + ", Question:" + query
                }
            ],
            temperature = 0,
            model="gpt-4o-mini",
        )
        res = r.choices[0].message.content
        results.append(res)
        print(res)
        
    except Exception as e:
        Errors += 1
        # Error_L.append(i)
        print(e)
    


```json
[19, 20, 43, 44, 47, 48, 66, 67, 502]
```
```json
[20, 19, 43, 44, 47, 27, 64, 88, 202, 521]
```
```json
[21, 104, 105, 106, 108, 111, 113, 115, 116, 129]
```
```json
[23, 22, 21, 239, 171, 172, 500, 501, 506, 491]
```
```json
[24, 25, 23, 26, 37, 300, 309, 347]
```
```json
[24, 25, 200, 301, 302, 400, 403, 404, 301, 302]
```
```json
[25, 24, 23, 301, 302, 278, 309, 301, 302, 347]
```
```json
[258, 259, 260, 26, 27, 44, 45, 46, 47, 284]
```
```json
[61, 27, 55, 215, 258, 259, 282, 283, 284, 286]
```
```json
[31, 30, 25, 37, 172, 173, 200, 399, 419, 107]
```
```json
[31, 30, 32, 25, 62, 352, 361, 385, 399, 278]
```
```json
[32, 39, 56, 62, 73, 198, 199, 373, 417, 31]
```
```json
[34, 35, 36, 32, 31, 39, 32, 73, 198, 199]
```
```json
[36, 35, 34, 32, 39, 37, 55, 57, 60, 73]
```
```json
[36, 35, 34, 39, 55, 57, 60, 32, 73, 200]
```
```json
[37, 39, 32, 31, 60, 64, 73, 198, 199, 373]
```
```json
[39, 32, 35, 36, 58, 73, 198, 199, 373, 374]
```
```json
[43, 44, 47, 48, 51, 56, 66, 1

In [7]:
import json
true_result = []
for res in results:
    temp = res.replace("`", "").replace("json", "").replace("\"", "")
    # print(temp)
    true_result.append(json.loads(temp))

top1 = 0
top3 = 0
top5 = 0
top10 = 0
# top20 = 0
# top50 = 0
# top100 = 0

page_distances = [] # the distances of the correct page and retrieved page
scores = [] # the score of the corresponding page and question
correct_rank = [] # the index of the correct page in the retrieved results.

for i in range(len(data)):
    page_number = data[i]['page']
    reranked = true_result[i]
    # retrived_page_rank = [list(data[i]['question_retrived_page_rank'][d].keys())[0] for d in range(len(data[i]['question_retrived_page_rank']))]
    # correct_page_rank = retrived_page_rank.index(page_number)
    # correct_rank.append(correct_page_rank)
    # page_distances.append(abs(page_number - retrived_page_rank[0]))
    # scores.append(cos_sim(data[i]['question_embedding'], data[i]['page_embedding']))
    
    if page_number in reranked[:1]: top1 += 1
    if page_number in reranked[:3]: top3 += 1
    if page_number in reranked[:5]: top5 += 1
    if page_number in reranked[:10]: top10 += 1
    # if page_number in retrived_page_rank[:20]: top20 += 1
    # if page_number in retrived_page_rank[:50]: top50 += 1
    # if page_number in retrived_page_rank[:100]: top100 += 1

In [8]:
print(top1/477)
print(top3/477)
print(top5/477)
print(top10/477)

0.5052410901467506
0.8406708595387841
0.9035639412997903
0.9832285115303984
